# 🎯 Lab 12: Model Selection & Evaluation
## Module 12 - Trade-offs: Latency, Cost, Accuracy

**Duration:** 20 minutes

**Objectives:**
- Compare model performance for banking tasks
- Evaluate with appropriate metrics
- Make cost-effective model choices

**Banking Scenario:** Select optimal model for document classification

---

In [ ]:
import os
import json
import time

# =============================================================================
# GOOGLE COLAB SETUP - Add these secrets (click 🔑 icon):
#   - AZURE_OPENAI_KEY: Your API key
#   - AZURE_OPENAI_ENDPOINT: https://xxx.openai.azure.com/
#   - AZURE_OPENAI_DEPLOYMENT: Your model deployment name
# =============================================================================

DEMO_MODE = False
client = None
MODEL_NAME = "gpt-4o"

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    try:
        MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except:
        pass
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print(f"✅ Credentials loaded. Model: {MODEL_NAME}")
    else:
        raise ValueError("Missing")
except Exception:
    print("⚠️ Running in DEMO MODE")
    DEMO_MODE = True

if not DEMO_MODE:
    from openai import AzureOpenAI
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        api_version="2024-06-01",
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    print("✅ Client ready")

## Part 1: Model Comparison

In [ ]:
MODELS = {
    "gpt-4o": {"cost_input": 5.0, "cost_output": 15.0},
    "gpt-4o-mini": {"cost_input": 0.15, "cost_output": 0.60}
}

def benchmark_model(model: str, prompt: str, expected: str) -> dict:
    """Benchmark a model on a single task"""
    start = time.time()
    
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=100
    )
    
    latency = time.time() - start
    output = response.choices[0].message.content
    tokens = response.usage.total_tokens
    
    # Calculate cost
    pricing = MODELS[model]
    cost = (response.usage.prompt_tokens * pricing["cost_input"] + 
            response.usage.completion_tokens * pricing["cost_output"]) / 1_000_000
    
    # Simple accuracy check
    correct = expected.lower() in output.lower()
    
    return {
        "model": model,
        "latency": latency,
        "tokens": tokens,
        "cost": cost,
        "correct": correct,
        "output": output[:100]
    }

print("✅ Benchmark function defined")

In [ ]:
# Test cases for banking document classification
test_cases = [
    {"prompt": "Classify this document: 'Application for $250,000 home loan at 6.5% for 30 years'", "expected": "mortgage"},
    {"prompt": "Classify this document: 'Request to dispute charge of $45.99 from unknown merchant'", "expected": "dispute"},
    {"prompt": "Classify this document: 'Inquiry about current CD rates and terms'", "expected": "inquiry"}
]

results = []
for model in MODELS.keys():
    print(f"\nTesting {model}...")
    for test in test_cases:
        result = benchmark_model(model, test["prompt"], test["expected"])
        results.append(result)
        print(f"  ✓ {test['expected']}: {result['latency']:.2f}s, ${result['cost']:.6f}")

## Part 2: Results Analysis

In [ ]:
# Aggregate results by model
from collections import defaultdict

stats = defaultdict(lambda: {"latency": [], "cost": [], "correct": 0, "total": 0})

for r in results:
    model = r["model"]
    stats[model]["latency"].append(r["latency"])
    stats[model]["cost"].append(r["cost"])
    stats[model]["correct"] += 1 if r["correct"] else 0
    stats[model]["total"] += 1

print("\n" + "="*60)
print("MODEL COMPARISON RESULTS")
print("="*60)
print(f"{'Model':<15} {'Avg Latency':<12} {'Avg Cost':<12} {'Accuracy':<10}")
print("-"*60)

for model, data in stats.items():
    avg_latency = sum(data["latency"]) / len(data["latency"])
    avg_cost = sum(data["cost"]) / len(data["cost"])
    accuracy = data["correct"] / data["total"]
    print(f"{model:<15} {avg_latency:.2f}s{'':<6} ${avg_cost:.6f}{'':<4} {accuracy:.0%}")

## Part 3: Cost Projection

In [ ]:
# Project costs at scale
DAILY_REQUESTS = 10000

print("\n" + "="*60)
print("COST PROJECTION (10,000 daily requests)")
print("="*60)

for model, data in stats.items():
    avg_cost = sum(data["cost"]) / len(data["cost"])
    daily = avg_cost * DAILY_REQUESTS
    monthly = daily * 30
    print(f"{model}: ${daily:.2f}/day, ${monthly:.2f}/month")

print("\n💡 Recommendation: Use GPT-4o-mini for classification tasks")
print("   Savings: ~97% with comparable accuracy")

---
## ✅ Lab 12 Complete!

**Key Takeaways:**
- GPT-4o-mini is 97% cheaper with similar accuracy for simple tasks
- Always benchmark before choosing a model
- Consider latency requirements for real-time applications